In [1]:
import numpy as np
import glob
import xarray as xr
import calendar
from pathlib import Path
import os

In [2]:
#Import files 
def get_all_files():
    root='MERRA2/'
    path = root+'hrly/TLML'
    print(path)
    files = glob.glob(path + "/*.nc")
    if len(files)==0:
        print("No files found")
    return files

In [3]:
files=get_all_files()
Path(files[0]).parent.exists()

MERRA2/hrly/TLML


True

In [3]:
def test_file_getter():
    files = get_all_files()
    num_leap_days= calendar.leapdays(1980, 2023)
    #print(len(files[1]))
    assert len(files) == 365 * 44 + num_leap_days
    print(xr.open_dataset(files[0]))
test_file_getter()

MERRA2/hrly/TLML
<xarray.Dataset> Size: 10MB
Dimensions:  (time: 24, lon: 576, lat: 180)
Coordinates:
  * time     (time) datetime64[ns] 192B 1996-12-19T00:30:00 ... 1996-12-19T23...
  * lon      (lon) float64 5kB -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat      (lat) float64 1kB 0.5 1.0 1.5 2.0 2.5 ... 88.0 88.5 89.0 89.5 90.0
Data variables:
    TLML     (time, lat, lon) float32 10MB ...
Attributes: (12/32)
    CDI:                               Climate Data Interface version 1.9.8 (...
    Conventions:                       CF-1
    History:                           Original file generated: Fri Oct 17 04...
    Comment:                           GMAO filename: d5124_m2_jan91.tavg1_2d...
    Filename:                          MERRA2_200.tavg1_2d_flx_Nx.19961219.nc4
    Institution:                       NASA Global Modeling and Assimilation ...
    ...                                ...
    RangeBeginningDate:                1996-12-19
    RangeBeginningTime:          

In [19]:
#Not used!!!
def save_variables1(arrs, file):

    root = 'MERRA2/'
    droot = os.path.join(root, "daily")
    
    tmin_dir = os.path.join(droot, "TMIN")
    tmax_dir = os.path.join(droot, "TMAX")
    
    if not os.path.exists(tmin_dir):
        os.makedirs(tmin_dir)  
    if not os.path.exists(tmax_dir):
        os.makedirs(tmax_dir)  
    base_file = os.path.basename(file)
    minfile = os.path.join(tmin_dir, file.replace(".TLML.", ".TMIN."))
    maxfile = os.path.join(tmax_dir, file.replace(".TLML.", ".TMAX."))
    

    arrs[0].name = "TMIN"
    arrs[0].attrs = {
        "long_name": "minimum daily temperature",
        "units": "K"
    }
    arrs[0].to_netcdf(minfile)
    
    arrs[1].name = "TMAX"
    arrs[1].attrs = {
        "long_name": "maximum daily temperature",
        "units": "K"
    }
    arrs[1].to_netcdf(maxfile)


In [24]:
def save_variables2(arrs, file):
    
    root = 'MERRA2/'
    droot = os.path.join(root, "daily")
    
    
    tmin_dir = os.path.join(droot, "TMIN")
    tmax_dir = os.path.join(droot, "TMAX")
    
    
    os.makedirs(tmin_dir, exist_ok=True)  
    os.makedirs(tmax_dir, exist_ok=True)  

    
    base_file = Path(file).name  
    
    
    minfile = os.path.join(tmin_dir, base_file.replace(".TLML.", ".TMIN."))
    maxfile = os.path.join(tmax_dir, base_file.replace(".TLML.", ".TMAX."))
    
    
    arrs[0].name = "TMIN"
    arrs[0].attrs = {
        "long_name": "minimum daily temperature",
        "units": "K"
    }
    arrs[0].to_netcdf(minfile)
    
    
    arrs[1].name = "TMAX"
    arrs[1].attrs = {
        "long_name": "maximum daily temperature",
        "units": "K"
    }
    arrs[1].to_netcdf(maxfile)

In [19]:
#Not used!!!
def save_variables(arrs, file):
    root='MERRA2/'
    droot=root+"/daily/"
    minfile=droot+"/TMIN/"+file.replace(".TLML.",".TMIN.")
    maxfile=droot+"/TMAX/"+file.replace(".TLML.",".TMAX.")
    arrs[0].name="TMIN"
    arrs[0].attrs={"long_name": "minimum daily temperature", "units":"K"}
    arrs[0].to_netcdf(minfile)
    arrs[1].name="TMAX"
    arrs[1].attrs={"long_name": "maximum daily temperature", "units":"K"}
    arrs[1].to_netcdf(maxfile)

In [20]:
def compute_min(temp):
    return temp.coarsen(time=24,coord_func="min").min("time")

In [21]:
def compute_max(temp):
    return temp.coarsen(time=24,coord_func="min").max("time")

In [25]:
def calc_min_max(file):
    temp=xr.open_dataset(file)['TLML']
    tmin=compute_min(temp)
    tmax=compute_max(temp)
    save_variables2([tmin, tmax], f)

In [26]:
files = get_all_files()
for f in files:
    calc_min_max(f)

MERRA2/hrly/TLML


In [1]:
#Compute metric threshold

In [2]:
def get_files_for_month(month):
    root='MERRA2/'
    path = root+'daily/TMAX'
    allfiles= glob.glob(path + "/*.nc")
    monthfiles=[]
    for file in allfiles:
        filename = os.path.basename(file)
        parts = filename.split(".")
        month_part = parts[3]
        if month_part[4:6] == month:
            monthfiles.append(file)
    return monthfiles

In [3]:
def test_files_for_month(month):
    files = get_files_for_month(month)
    num_days_in_month= calendar.monthrange(1981,int(month))[1] #Non leap year 
    if month=='02':
        num_days_in_months= 44* num_days_in_month + calendar.leapdays(1980, 2023)
        assert len(files) == 44 * num_days_in_months, len(files)
    #print(len(files[1]))
    else: 
        assert len(files) == 44 * num_days_in_month, num_days_in_month
    print(xr.open_dataset(files[0]))
test_files_for_month('07')

<xarray.Dataset> Size: 421kB
Dimensions:  (time: 1, lon: 576, lat: 180)
Coordinates:
  * time     (time) datetime64[ns] 8B 2003-07-23T00:30:00
  * lon      (lon) float64 5kB -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat      (lat) float64 1kB 0.5 1.0 1.5 2.0 2.5 ... 88.0 88.5 89.0 89.5 90.0
Data variables:
    TMAX     (time, lat, lon) float32 415kB ...


In [14]:
lon_min = -30
lon_max = 40
lat_min = 30
lat_max = 80

def compute_metric_threshold(month, threshold=0.95):
    files = get_files_for_month(month)
    dataset = xr.open_mfdataset(files)
    tempmax = dataset['TMAX']
    region_dict = {"lon": slice(lon_min, lon_max), "lat": slice(lat_min, lat_max)}
    regiontmax = tempmax.sel(region_dict)
    regiontmax.load()
    TMAXP95=regiontmax.quantile(threshold, dim='time')
    TMAXP95 = TMAXP95.expand_dims(month=[month])
    save_metric_thr_variable([TMAXP95],files[0])

In [15]:
def save_metric_thr_variable(arrs,file):
    root='MERRA2/'
    droot=os.path.join(root, "stats")
    tmaxp95dir = os.path.join(droot, "TMAXP95")
    os.makedirs(tmaxp95dir, exist_ok=True)
    base_file = Path(file).name 
    tmaxp95file = os.path.join(tmaxp95dir, base_file.replace(".TMAX.", ".TMAXP95."))
    arrs[0].name="TMAXP95"
    arrs[0].attrs={"long_name": "95%ile daily temp for month", "units":"K"}
    arrs[0].to_netcdf(tmaxp95file)

In [16]:
compute_metric_threshold('01')

In [17]:
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
for month in months:
    compute_metric_threshold(month)

In [ ]:
def testTMthr():
    filegreater=[]
    